In [1]:
import duckdb
import pandas as pd
import os
from lotus.dtype_extensions import ImageArray
from lotus.types import CascadeArgs, ProxyModel
import pandas as pd
from join_optimizer.join_optimizer.productDisplayName_index.evaluate import *


parquet_path_sampeled = 'fashion_product_images_001/styles.parquet'
parquet_path = 'fashion_product_images_001/styles.parquet'
details_path = 'fashion_product_images_001/styles_details.parquet'
sample_size = 1
df = duckdb.query(f"""
with images as (
    SELECT *
    FROM parquet_scan('{parquet_path}')
    USING SAMPLE {sample_size * 100} PERCENT (reservoir, 80)
    )
    select
     images.id ,images.subcategory, images.articletype, images.basecolour, details.price, images.productDisplayName, styleimages.default.imageURL
    -- *
    from images, parquet_scan('{details_path}') details
    where images.id = details.id
    -- and details.price <1000

""").to_df()



/home/dhia/miniconda3/envs/lotus/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/dhia/miniconda3/envs/lotus/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
image_file_names = df["id"]
image_URLs = df["imageURL"]
image_paths = [os.path.join("fashion_product_images/images_resized", str(image) + ".jpg") for image in image_file_names]
df2 = pd.DataFrame({"image": ImageArray(image_URLs), "label": image_file_names, "image_path": image_paths, "image_URLs": image_URLs , "articleType": df["articleType"], "baseColour": df["baseColour"]})
merged_df = pd.merge(df, df2,  left_on='id', right_on='label')
merged_df.columns = ['_' + col for col in merged_df.columns]


In [ ]:
x.p


In [ ]:

import lotus
from lotus.models import LM, SentenceTransformersRM
from lotus.types import CascadeArgs
from lotus.vector_store import FaissVS
lm = LM(model="gemini/gemini-2.0-flash-lite")
rm = SentenceTransformersRM(model="intfloat/e5-base-v2")
vs = FaissVS()
lotus.settings.configure(lm=lm, rm=rm, vs=vs)

lm.print_total_usage()

filtered_df2 = (df2
                .sem_filter("the content  of {image} shows a white t-shirt")
                )
lm.print_total_usage()


In [ ]:
image_paths_big = [os.path.join("fashion_product_images/images", str(image) + ".jpg") for image in image_file_names if str(image)[10] == 1]
df2_big = pd.DataFrame({"image": ImageArray(image_paths_big), "label": image_file_names, "image_path": image_paths})

lm_2 = LM(model="gemini/gemini-2.0-flash-lite")
rm = SentenceTransformersRM(model="intfloat/e5-base-v2")
vs = FaissVS()
lotus.settings.configure(lm=lm_2, rm=rm, vs=vs)

lm_2.print_total_usage()

filtered_df2_big = (df2_big
                .sem_filter("the content  of {image} shows a white t-shirt")
                )
lm_2.print_total_usage()


In [ ]:
filtered_df2


In [ ]:
filtered_df = (df
                .sem_filter("The {articleType}, {baseColour} and {productDisplayName} might show a t-shirt that might be white")
                )
lm.print_total_usage()



In [ ]:
filtered_df


In [ ]:
expr = "Does {productDisplayName} exactly and precisely match the {image}?"


cascade_args = CascadeArgs(recall_target=0.8, precision_target=0.8)
res = filtered_df.sem_join(filtered_df2, expr, return_stats=True, strategy="zs-cot")


print(f"Joined {df.shape[0]} rows from df1 with {filtered_df2.shape[0]} rows from df2")
# print(f"    Join cascade took {stats['join_resolved_by_large_model']} LM calls")
# print(f"    Helper resolved {stats['join_resolved_by_helper_model']} LM calls")
# print(f"Join cascade used {stats['total_LM_calls']} LM calls in total")
# print(f"Naive join would require {df.shape[0]*df2.shape[0]} LM calls")
res

In [ ]:
lm.print_total_usage()


In [ ]:
expr = "given {productDisplayName}, which {image} does more exactly and precisely match?"


ranked, stats = res.sem_topk(
    expr,
    K = 1,
    group_by=["productDisplayName"],
    method="quick",
    return_stats=True
)
ranked

In [ ]:
lm.print_total_usage()


# SEM INDEX


#### Creating the index

In [3]:
from lotus.vector_store import FaissVS
import lotus
from lotus.models import LM, SentenceTransformersRM

gpt_4o_mini = LM("gpt-4o-mini")
gpt_4o = LM("gpt-4o")

# CLIP embedding model – works for both text & image
# rm  = SentenceTransformersRM(model="clip-ViT-B-32")
rm  = SentenceTransformersRM(model="clip-ViT-L-14", max_batch_size=32)

lotus.settings.configure(lm=gpt_4o, helper_lm=gpt_4o_mini, rm=rm, vs=FaissVS())

2025-08-12 08:17:42,388 - INFO - Use pytorch device_name: cpu
2025-08-12 08:17:42,389 - INFO - Load pretrained SentenceTransformer: clip-ViT-L-14
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [30]:
merged_df = merged_df.sem_index("_image", index_dir="image_100_index")
df = df.sem_index("productDisplayName", index_dir="productDisplayName_100_index")



2025-08-10 09:26:38,638 - INFO - Use pytorch device_name: cpu
2025-08-10 09:26:38,641 - INFO - Load pretrained SentenceTransformer: clip-ViT-L-14
2025-08-10 09:26:40,388 - WARNING - Do not reset the dataframe index to ensure proper functionality of get_vectors_from_index
2025-08-10 09:34:27,886 - WARNING - Do not reset the dataframe index to ensure proper functionality of get_vectors_from_index


In [4]:
merged_df = merged_df.load_sem_index("_image", index_dir="image_100_index")
df = df.load_sem_index("productDisplayName", index_dir="productDisplayName_100_index")
# merged_df = merged_df.sem_map("Describe the product in the {_image} in 15 words.", suffix = "_description")


##### Binary search filter

In [21]:

cascade_args = CascadeArgs(
    recall_target=0.9,
    precision_target=0.9,
    sampling_percentage=0.1,
    proxy_model=ProxyModel.EMBEDDING_MODEL,
)

merged_df_filtered =  merged_df.sem_filter("{_image} is a white T-shirt",cascade_args=cascade_args, return_stats=True, find_top_k=True)

Filtering: 100%|██████████ 1/1 LM calls [00:03<00:00,  3.91s/it]
Filtering: 100%|██████████ 1/1 LM calls [00:01<00:00,  1.81s/it]
Filtering: 100%|██████████ 1/1 LM calls [00:02<00:00,  2.61s/it]
Filtering: 100%|██████████ 1/1 LM calls [00:07<00:00,  7.80s/it]
Filtering: 100%|██████████ 1/1 LM calls [00:08<00:00,  8.88s/it]
Filtering: 100%|██████████ 1/1 LM calls [00:07<00:00,  7.12s/it]
Filtering: 100%|██████████ 1/1 LM calls [00:05<00:00,  5.98s/it]
Filtering: 100%|██████████ 1/1 LM calls [00:09<00:00,  9.22s/it]
Filtering: 100%|██████████ 1/1 LM calls [00:03<00:00,  3.65s/it]
Filtering: 100%|██████████ 1/1 LM calls [00:02<00:00,  2.44s/it]
Filtering: 100%|██████████ 1/1 LM calls [00:08<00:00,  8.04s/it]


In [22]:
from join_optimizer.join_optimizer.productDisplayName_index.evaluate import *

metrics, FP, FN = evaluate_filter(
    merged_df=merged_df,
    filtered_df=merged_df_filtered,
    article_type='Tshirts',
    base_colour="White"
)
print(metrics)

{'TP': 12, 'FP': 3, 'FN': 2, 'precision': 0.8, 'recall': 0.8571428571428571, 'f1': 0.8275862068965518}


#### full LLM calls

In [18]:
import sys
import importlib

importlib.reload(sys.modules['lotus.sem_ops.cascade_utils'])
importlib.reload(sys.modules['lotus.sem_ops.sem_filter'])

/home/dhia/lotus_/lotus/lotus/sem_ops/sem_filter.py:190: UserWarning: registration of accessor <class 'lotus.sem_ops.sem_filter.SemFilterDataframe'> under name 'sem_filter' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @pd.api.extensions.register_dataframe_accessor("sem_filter")


<module 'lotus.sem_ops.sem_filter' from '/home/dhia/lotus_/lotus/lotus/sem_ops/sem_filter.py'>

In [34]:
cascade_args = CascadeArgs(
    recall_target=0.95,
    precision_target=0.9,
    sampling_percentage=0.1,
    proxy_model=ProxyModel.EMBEDDING_MODEL,
    cascade_IS_weight=1,
    cascade_num_calibration_quantiles = 100,
    failure_probability=0.1,
    cascade_IS_random_seed=332114,
    cascade_IS_max_sample_range=444

)

merged_df_filtered_big =  merged_df.sem_filter("{_image} is a sock", cascade_args=cascade_args ,return_stats=False, find_top_k=False)




Running oracle for threshold learning:   0%|           0/44 LM calls [00:00<?, ?it/s]

Running oracle for threshold learning: 100%|██████████ 44/44 LM calls [00:08<00:00,  4.92it/s]
2025-08-12 08:46:04,434 - INFO - Sample recall: 0.0
2025-08-12 08:46:04,436 - INFO - Sample precision: 0.0
2025-08-12 08:46:04,439 - INFO - Learned cascade thresholds: (1.0, 0)
2025-08-12 08:46:04,442 - INFO - Num routed to smaller model: 0


Running predicate evals with oracle LM:   0%|           0/444 LM calls [00:00<?, ?it/s]2025-08-12 08:46:16,630 - INFO - Retrying request to /chat/completions in 0.463313 seconds
Running predicate evals with oracle LM:   0%|           0/444 LM calls [05:29<?, ?it/s]


Running predicate evals with oracle LM: 100%|██████████ 444/444 LM calls [00:38<00:00, 11.55it/s]


In [36]:
metrics, FP, FN = evaluate_filter(
    merged_df=merged_df,
    filtered_df=merged_df_filtered_big,
    article_type='Socks',
    base_colour=None
)
print(metrics)

{'TP': 9, 'FP': 0, 'FN': 0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}


#### Just sim_search

In [13]:
# sem_search with similarity scores returned
sim_df_with_scores = merged_df.sem_search(
    "_image",
    "_image is a white tshirt",
    K=25,
    return_scores=True,
    suffix="_similarity_score"  # This will create a column named "vec_scores_similarity_score"
)


#### Joins

In [18]:
res_sim_join = merged_df_filtered.sem_sim_join(df, left_on='_image', right_on='productDisplayName', K=1, keep_index=True)

NameError: name 'merged_df_filtered' is not defined

In [1]:
expr = "Does {productDisplayName} exactly and precisely match the {_image}?"
from lotus.types import CascadeArgs, ProxyModel

cascade_args = CascadeArgs(
    recall_target=0.9,
    precision_target=0.75,
    sampling_percentage=0.04,
    proxy_model=ProxyModel.EMBEDDING_MODEL,
)

res = merged_df_filtered.sem_join(df, expr,cascade_args=cascade_args, return_stats=True)

# print(f"Joined {df.shape[0]} rows from df1 with {filtered_df2.shape[0]} rows from df2")
# print(f"    Join cascade took {stats['join_resolved_by_large_model']} LM calls")
# print(f"    Helper resolved {stats['join_resolved_by_helper_model']} LM calls")
# print(f"Join cascade used {stats['total_LM_calls']} LM calls in total")
# print(f"Naive join would require {df.shape[0]*df2.shape[0]} LM calls")


/home/dhia/miniconda3/envs/lotus/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/dhia/miniconda3/envs/lotus/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


NameError: name 'merged_df_filtered' is not defined

In [ ]:
res = 0

In [ ]:

expr = "given {productDisplayName}, which {_image} does more exactly and precisely match?"

ranked, stats = res[0].sem_topk(
    expr,
    K=1,
    group_by=["productDisplayName"],
    method="quick",
    return_stats=True
)
ranked


In [58]:
join_topk = ranked.query('_id == id')
sim_1 = res_sim_join.query('_id == id')